In [1]:
from policyengine_uk import Microsimulation
from policyengine_uk.data import UKDataset
from policyengine_uk_data.datasets.frs import create_frs

frs = UKDataset("policyengine_uk_data/storage/frs_2022.h5")

In [2]:
from policyengine_uk_data.datasets.local_areas.constituencies.calibrate import calibrate

frs_calibrated = calibrate(frs)

IndexError: single positional indexer is out-of-bounds